# Development Notebook:

## Get Device IDs Dynamically

In [23]:
lfile = "/Users/shanto/Google Drive/Shared drives/Labber/Data/2023/02/Data_0202/TWPA_Tune_Up_NBR26.hdf5"

In [22]:
lfile2 = "/Users/shanto/Google Drive/Shared drives/Labber/Data/2023/02/Data_0208/JPA_pumppower_flux_SA_NBR26.hdf5"

In [28]:
lf = Labber.LogFile(lfile2)

In [29]:
lf.getLogChannels()[0]["name"]

'HP Spectrum Analyzer - Signal'

In [30]:
lf.getStepChannels()[1]["name"]

'10002F25 - Power'

In [31]:
lf.getStepChannels()

[{'name': 'Counter - Value 1',
  'unit': '',
  'values': array([1., 2., 3.]),
  'complex': False,
  'vector': False},
 {'name': '10002F25 - Power',
  'unit': 'dBm',
  'values': array([14.  , 14.25, 14.5 , 14.75, 15.  , 15.25, 15.5 , 15.75, 16.  ,
         16.25, 16.5 , 16.75, 17.  , 17.25, 17.5 , 17.75, 18.  ]),
  'complex': False,
  'vector': False},
 {'name': 'Vikram - Source current',
  'unit': 'A',
  'values': array([-0.0004 , -0.00041, -0.00042, -0.00043, -0.00044, -0.00045,
         -0.00046, -0.00047, -0.00048, -0.00049, -0.0005 , -0.00051,
         -0.00052, -0.00053, -0.00054, -0.00055]),
  'complex': False,
  'vector': False}]

## Create Gain Profile Plot

## Create SNR Improvement Plot

## SNR Region Extraction:

In [1]:
import Labber
import os
import sys
from fitTools.utilities import Watt2dBm, dBm2Watt, VNA2dBm

from twpa_tune_up_helper_functions import *

/Users/shanto/anaconda3/envs/lfl_qp/lib/python3.9/site-packages/qtpy/__init__.py:214: RuntimeWarning: Selected binding "pyqt5" could not be found, using "pyqt6"
  warnings.warn('Selected binding "{}" could not be found, '


In [ ]:
labber_data_file = str(input("Labber File Location: "))
repeated = int(input("Number of Repeations: "))
power_range = int(input("Number of Points for Power: "))
freq_range = int(input("Number of Points for Frequency: "))

std_highSNR = 1.15 # cut off point for determining high SNR
cutOff_around_SA_peak = 10e3 # Hz

lf = Labber.LogFile(labber_data_file)

power_channel_name = "10002F25 - Power"
freq_channel_name = "10002F25 - Frequency"
SA_channel_name = 'HP Spectrum Analyzer - Signal'

pump_power = lf.getData(name = power_channel_name)
pump_freq = lf.getData(name = freq_channel_name)

signal = lf.getData(name = SA_channel_name)
linsig = dBm2Watt(signal)

SAxdata, SAydata = lf.getTraceXY(y_channel=SA_channel_name) # gives last trace from SA

plt.rcParams['savefig.facecolor']='white'

get_SNR_space_plot(signal,repeated, freq_range, power_range, pump_freq,
                   pump_power, SAxdata, cutOff=cutOff_around_SA_peak,
                   title="TWPA Tune Up", xlabel='Pump Power (dBm)', 
                   ylabel='Pump Frequency (Hz)', zlabel='SNR', 
                   fig_type=".png", path="figures")




In [ ]:
def get_high_SNR_regions(signal,repeated, freq_range, power_range,pump_freq, pump_power, SAxdata, cutOff=10e3, std_highSNR=1.75):
    average_signal = get_average_of_N_traces(signal,repeated)
    average_lin_signal = dBm2Watt(average_signal)

    pump_freqs = np.linspace(pump_freq[0][0],pump_freq[-1][-1],freq_range)
    pump_powers = np.linspace(pump_power[0][0],pump_power[-1][-1],power_range)

    SNRs, max_signals, noise_floors = calculate_SNRs(average_lin_signal,SAxdata,cutOff)
    SNRs_reshaped = np.reshape(SNRs, (freq_range,power_range))

    meanSNR = np.mean(SNRs_reshaped)
    region = get_config_for_high_SNR(SNRs_reshaped,x=pump_powers, y=pump_freqs,std_dev=std_highSNR)
    std_message = f"Region of High SNR\n[i.e SNR > mean(SNR) * std_dev(SNR)]\nmean(SNR) = {meanSNR:.3f}, std_dev(SNR) = {std_highSNR:.2f}"
    create_heatmap(region, pump_powers, pump_freqs, title = std_message, xlabel='Pump Power (dBm)', ylabel='Pump Frequency (Hz)', zlabel='SNR',)
    print_coordinates(get_coordinates(pump_powers, pump_freqs,region))
    return get_coordinates(pump_powers, pump_freqs,region)

def get_config_for_high_SNR(arr,x,y,std_dev=2):
    mean = np.mean(arr)
    standard_deviation = np.std(arr)
    distance_from_mean = abs(arr - mean)
    min_deviations = std_dev
    high_values = mean + min_deviations * standard_deviation

    arr[arr < high_values] = np.nan
    return arr


def get_coordinates(x,y,z):
    mask = np.isfinite(z)
    xx,yy = np.array(np.meshgrid(x, y))
    masked_x = mask*xx
    masked_y = mask*yy
    x = masked_x[masked_x>0]
    y = masked_y[masked_y>0]
    z = z[np.isfinite(z)]
    return np.array((x,y,z)).T

def print_coordinates(arr):
    for i in range(arr.shape[0]):
        print(f"SNR = {arr[i][2]:.3f} for Power = {arr[i][1]}and Frequency = {arr[i][0]}")

In [ ]:
region= get_high_SNR_regions(signal,repeated, freq_range, power_range, pump_freq,
                         pump_power, SAxdata, cutOff=cutOff_around_SA_peak,
                         std_highSNR=2)

In [ ]:
region.shape[0]

## SNR Methods

In [ ]:
import Labber
import os
import sys
from fitTools.utilities import Watt2dBm, dBm2Watt, VNA2dBm

from twpa_tune_up_helper_functions import *


In [ ]:
def calculate_mean_SNR_from_Labber_file(labber_data_file5, cutOff = 10e3):
    """
    """
    lf = Labber.LogFile(labber_data_file)
    SA_channel_name = 'HP Spectrum Analyzer - Signal'


    signal = lf.getData(name = SA_channel_name)
    linsig = dBm2Watt(signal)

    SAxdata, SAydata = lf.getTraceXY(y_channel=SA_channel_name, entry=0) # gives last trace from SA
    
    snrs = 0
    for i in range(len(signal)):
        snrs += get_signal_stats(dBm2Watt(signal[i]), SAxdata, cutOff)[0]

    snr_mean = snrs / len(signal)
    return snr_mean

**Testing SNR Code:**

In [ ]:
calculate_mean_SNR_from_Labber_file